# VaR Monte Carlo Evaluation with Python

In this notebook we evaluate the Value at Risk (VaR) of a certain stocks with the use of Monte Carlo methods on Python.

Let us have an asset  with price $S$ an compute it's daily returns. For simplicity,  we will work on the assumption that these returns follow a normal distribution, with mean $\mu$ and standard deviation $\sigma$. Then the following formula for calculating tomorrow's VaR:
$$
\text{VaR} = S\left(\mu -\sigma\,\alpha(1-c)\right)\,,
$$
where $c$ is the confidence level we choose to work with, and $c(x)$ is the percent point function for the standard normal distribution. If we want to compute the VaR over a period of time $T$, then the formula has to be modified to:
$$
\text{VaR} = S\left(\mu\,T-\sigma\,\sqrt{T}\alpha(1-c)\right)\,,
$$

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from pandas_datareader import data as pdr
import yfinance as yf 
yf.pdr_override() 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
plt.style.use('fivethirtyeight')

We will apply these formulas for the case of Apple stocks, APPL. We choose the time period between 2015-10-10 and 2018-01-01- We compute all the relevant quantities. 

In [12]:
def VaR(stock, s, start, end,c ,period):
    
    data = pdr.get_data_yahoo(stock, start=start, end=end)['Adj Close']
    returns = (data/data.shift(1))-1
    returns.dropna(inplace=True)
    
    mu = returns.mean()
    sigma =returns.std()
    alpha=norm.ppf(1-c)
    
    var = s*(mu*period - sigma*alpha*(period**0.5))
    
    print('\n')
    print('Stock: {}'.format(stock[0]))
    print('Mu over {} days:'.format(period), mu*period)
    print('Sigma over {} days:'.format(period), sigma*np.sqrt(period))
    
    print('VaR for {} with confidence {} over period of {} days: '.format(stock[0],c,period), var)

We try with some stocks and time periods.

In [20]:
stock = ['^GSPC']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=1000
c=0.95
period =1.0

VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: ^GSPC
Mu over 1.0 days: 0.00037721678718513716
Sigma over 1.0 days: 0.007772088045011574
VaR for ^GSPC with confidence 0.95 over period of 1.0 days:  13.161163997008602


In [18]:
stock = ['^GSPC']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =252.0

VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: ^GSPC
Mu over 252.0 days: 0.09505863037065457
Sigma over 252.0 days: 0.12337807280879283
VaR for ^GSPC with confidence 0.95 over period of 252.0 days:  29.799750091648026


In [19]:
stock = ['AAPL']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =1.0

VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: AAPL
Mu over 1.0 days: 0.0007561794971024003
Sigma over 1.0 days: 0.014417074284163165
VaR for AAPL with confidence 0.95 over period of 1.0 days:  2.447015642343698


In [21]:
stock = ['AAPL']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =252

VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: AAPL
Mu over 252 days: 0.19055723326980487
Sigma over 252 days: 0.22886395913424173
VaR for AAPL with confidence 0.95 over period of 252 days:  56.70049465302359


In [22]:
stock = ['GE']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =252

VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: GE
Mu over 252 days: -0.0631523849211899
Sigma over 252 days: 0.1995439407831885
VaR for GE with confidence 0.95 over period of 252 days:  26.506818981222747


# Monte Carlo Approach

We now follow a Monte Carlo approach to VaR. We know that a given stock will follow a Wiener process. Then, given a period $T$, the stock price will follow the equation:
$$
S(T) = S(0)\exp \left(\left(\mu - \frac{1}{2}\sigma^{2}\right)T + \sigma \sqrt{T}\,z\right)\,,
$$
where $\mu$ and$\sigma$ can be obtained from a stock's historical information, and $z$ is a random number following a normal standard distibuition $N(0,1)$.

The idea then is to use Monte Carlo methods to produce a large collection of stock prices the last equation. Having these collection, we determine the percentile corresponding to the confidence level we want to work with: for example, if we have a confidence level $c = 95 \%$, we have to find the lowest 5 percentile $S_{1-c}$. Then, the VaR will be given by:
$$
\text{VaR}_{c} = S(0)-S_{1-c}\,.
$$

In [59]:
def VaR_MC(stock, S, start, end,c ,period, iterations):
    
    data = pdr.get_data_yahoo(stock, start=start, end=end)['Adj Close']
    returns = (data/data.shift(1))-1
    returns.dropna(inplace=True)
    
    mu = returns.mean()
    sigma =returns.std()
    
    z = np.random.normal(0, 1, [1, iterations])
    
    ST = S*np.exp(period*(mu - 0.5*sigma**2)+sigma*np.sqrt(period)*z)
    
    ST = np.sort(ST)
    
    Spc = np.percentile(ST,(1-c)*100)
    
    var = S - Spc
    
    
    print('\n')
    print('Stock: {}'.format(stock[0]))
    print('Mu over {} days:'.format(period), mu*period)
    print('Sigma over {} days:'.format(period), sigma*np.sqrt(period))
    
    print('Monte Carlo VaR for {} with confidence {} over period of {} days: '.format(stock[0],c,period), var)

We compute the Monte Carlo VaR for various stocks and compare with the formula computation. We try 10 million iterations.

In [65]:
stock = ['AAPL']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =1
iterations = 10000000


VaR_MC(stock, s, start, end,c ,period, iterations)

[*********************100%***********************]  1 of 1 downloaded


Stock: AAPL
Mu over 1 days: 0.0007561794971024003
Sigma over 1 days: 0.014417074284163165
Monte Carlo VaR for AAPL with confidence 0.95 over period of 1 days:  2.279317065330659


In [62]:
stock = ['AAPL']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =1
#iterations = 100000


VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: AAPL
Mu over 1 days: 0.0007561794971024003
Sigma over 1 days: 0.014417074284163165
VaR for AAPL with confidence 0.95 over period of 1 days:  2.447015642343698


We also check for 1 year of trading:

In [70]:
stock = ['AAPL']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =252
iterations = 10000000


VaR_MC(stock, s, start, end,c ,period, iterations)

[*********************100%***********************]  1 of 1 downloaded


Stock: AAPL
Mu over 252 days: 0.19055723326980487
Sigma over 252 days: 0.22886395913424173
Monte Carlo VaR for AAPL with confidence 0.95 over period of 252 days:  19.116284626472023


In [66]:
stock = ['AAPL']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =252
#iterations = 100000


VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: AAPL
Mu over 252 days: 0.19055723326980487
Sigma over 252 days: 0.22886395913424173
VaR for AAPL with confidence 0.95 over period of 252 days:  56.70049465302359


In [78]:
stock = ['^GSPC']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =1
iterations = 10000000


VaR_MC(stock, s, start, end,c ,period, iterations)

[*********************100%***********************]  1 of 1 downloaded


Stock: ^GSPC
Mu over 1 days: 0.00037721678718513716
Sigma over 1 days: 0.007772088045011574
Monte Carlo VaR for ^GSPC with confidence 0.95 over period of 1 days:  1.236261705139313


In [76]:
stock = ['^GSPC']
start = pd.to_datetime('2015-01-01') 
end = pd.to_datetime('2018-01-01')
s=100
c=0.95
period =1
iterations = 1000000000


VaR(stock, s, start, end,c ,period)

[*********************100%***********************]  1 of 1 downloaded


Stock: ^GSPC
Mu over 1 days: 0.00037721678718513716
Sigma over 1 days: 0.007772088045011574
VaR for ^GSPC with confidence 0.95 over period of 1 days:  1.3161163997008603


So we see that for short periods there is good agreement between the different VaR computations. For longer periods, the formula for VaR seems to over estimate the VaR, but within the same order of magnitude as for the Monte Carlo simulation. 